# Compare building height and population

1. Stack population and builidng heights
2. Create 2o population layer describing 3x3 population sum
3. Calculate service access



In [144]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../../GOST")

import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.misc as misc
import GOSTRocks.osmMisc as osm_misc

In [29]:
wp_vrt = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/WP_2020_100m.vrt"
wp = rasterio.open(wp_vrt)
inWSF_folder = "/home/public/Data/PROJECTS/LEI/DLR_V2"
out_folder = "/home/wb411133/data/Projects/HeightComparison"

#Get a list of cities
allCities = {}
for root, dirs, files in os.walk(inWSF_folder):
    for f in files:
        if f[-4:] == ".tif":
            cityName = f.split("_")[0]
            if not cityName in allCities.keys():
                allCities[cityName] = [os.path.join(root, f)]
            else:
                allCities[cityName].append(os.path.join(root, f))

In [30]:
city = "Dhaka"
cur_city = allCities[city]
cur_folder = os.path.join(out_folder, city)
if not os.path.exists(cur_folder):
    os.makedirs(cur_folder)

In [63]:
inH = rasterio.open(cur_city[0])
# extract WP data from current extent and standardize extent and resolution to height data
bounds = box(*inH.bounds)
wp_data, out_transform = mask(wp, shapes=[bounds], crop=True)
wp_data[wp_data < 0] = 0
# sum filter
def sum(P):    
    return(P.sum())

wp_data[0,:,:] = generic_filter(wp_data[0,:,:], sum, (3,3))
meta = inH.meta.copy()
meta.update(width=wp_data.shape[2], 
            height=wp_data.shape[1],
            transform=out_transform,
           dtype = wp_data.dtype)
wp_file = os.path.join(cur_folder, "wp_2020.tif")
wp_file_reproj = os.path.join(cur_folder, "wp_2020_re.tif")
wp_file_reproj_smooth = os.path.join(cur_folder, "wp_2020_re_smooth.tif")

with rasterio.open(wp_file, 'w', **meta) as out:
    out.write(wp_data)
    
rMisc.standardizeInputRasters(rasterio.open(wp_file), inH, wp_file_reproj)

# Accessibility Analysis

In [147]:
speed_dict = {
   'residential': 20,  # kmph
   'primary': 40,
   'primary_link':35,
   'motorway':50,
   'motorway_link': 45,
   'trunk': 40,
   'trunk_link':35,
   'secondary': 30,
   'secondary_link':25,
   'tertiary':30,
   'tertiary_link': 25,
   'unclassified':20,
   'living_street':10,
   'service':10
}

def get_speed(x, s_dict):
    try: 
        
        speed = s_dict[x]
    except:
        if type(x) == list:
            try:
                speed = s_dict[x[0]]
            except:
                speed = 5
        else:
            speed=5
    return(speed)

def get_nodes(b, tags):
    nodes = osm.node_query(b[1], b[0], b[3], b[2], tags=tags)
    nodes_geom = [Point(x) for x in zip(nodes['lon'], nodes['lat'])]
    nodes_df = gpd.GeoDataFrame(nodes[['amenity','lat','lon']], geometry=nodes_geom, crs={'init':'epgs:4326'})
    return(nodes_df)

In [ ]:
sel_graph = ox.graph_from_bbox(b[3], b[1], b[2], b[0], retain_all=True)
sel_roads = gn.edge_gdf_from_graph(sel_graph)
sel_roads['speed'] = sel_roads['highway'].apply(lambda x: get_speed(x, speed_dict))

In [84]:
amenities = ['toilets', 'washroom', 'restroom']
toilets_tags = '"amenity"~"{}"'.format('|'.join(amenities))
toilets = get_nodes(inH.bounds, toilets_tags)

amenities = ['water_points', 'drinking_water', 'pumps', 'water_pumps', 'well']
water_tags = '"amenity"~"{}"'.format('|'.join(amenities))
water_points = get_nodes(inH.bounds, water_tags)
    
amenities = ['supermarket', 'convenience', 'general', 'department_stores', 'wholesale', 'grocery', 'general']
shp_tags = '"shop"~"{}"'.format('|'.join(amenities))
shops = get_nodes(inH.bounds, shp_tags)


In [167]:
# create a copy of inH with value set to slowest walking speed
distance_data = np.zeros(inH.shape) + 5
# burn the speeds into the distance_data using the road network 
sel_roads = sel_roads.sort_values(['speed'])
shapes = ((row['geometry'], row['speed']) for idx, row in sel_roads.iterrows())
speed_image = features.rasterize(shapes, out_shape=inH.shape, transform=inH.transform, fill=5)
# convert to a version that claculates the seconds to cross each cell
traversal_time = 100 / (speed_image * 1000 / (60 * 60)) # km/h --> m/s * resolution of image in metres

In [168]:
# save traversal image (burned in road network)
road_network = os.path.join(out_folder, "road_network.tif")
meta.update(dtype=distance_image.dtype)
with rasterio.open(road_network, 'w', **meta) as out:
    out.write_band(1, distance_image)

In [179]:
# create skimage graph
mcp = skimage.graph.MCP_Geometric(traversal_time)
# iterate through amenity
for amenity in [toilets, water_points, shops]:
    costs, traceback = mcp.find_costs(list(set([inH.index(x.x, x.y) for x in amenity['geometry']])))
    amenity_name = amenity.iloc[0]['amenity']
    print("Processing %s" % amenity_name)
    amenity.to_file(os.path.join(out_folder, "%s.shp" % amenity_name))
    distance_raster = os.path.join(out_folder, '%s_distance_roads.tif' % amenity_name)
    meta.update(dtype=costs.dtype)
    with rasterio.open(distance_raster, 'w', **meta) as out:
        out.write_band(1, costs)

Processing toilets
Processing drinking_water
Processing nan
